# 数据可视化与探索性数据分析

本笔记本展示如何使用数学编程技术进行数据可视化和探索性数据分析。

## 1. 数据准备

首先加载必要的库并准备示例数据集。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 设置绘图风格
sns.set_style("whitegrid")
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 生成模拟数据集
np.random.seed(42)
n_samples = 200

# 创建学生成绩数据
data = {
    '学号': range(1, n_samples + 1),
    '数学成绩': np.random.normal(75, 12, n_samples).clip(0, 100),
    '物理成绩': np.random.normal(70, 15, n_samples).clip(0, 100),
    '化学成绩': np.random.normal(72, 13, n_samples).clip(0, 100),
    '英语成绩': np.random.normal(68, 14, n_samples).clip(0, 100),
    '学习时间': np.random.normal(20, 5, n_samples).clip(5, 40),
    '出勤率': np.random.uniform(0.7, 1.0, n_samples)
}

# 添加相关性
data['数学成绩'] = data['数学成绩'] + 0.5 * data['学习时间']
data['物理成绩'] = data['物理成绩'] + 0.4 * data['学习时间']
data['化学成绩'] = data['化学成绩'] + 0.3 * data['学习时间']

# 创建DataFrame
df = pd.DataFrame(data)

# 计算平均分
df['平均分'] = df[['数学成绩', '物理成绩', '化学成绩', '英语成绩']].mean(axis=1)

# 根据平均分划分等级
def get_grade(score):
    if score >= 90:
        return 'A'
    elif score >= 80:
        return 'B'
    elif score >= 70:
        return 'C'
    elif score >= 60:
        return 'D'
    else:
        return 'F'

df['等级'] = df['平均分'].apply(get_grade)

print("数据集形状:", df.shape)
print("\n前5行数据:")
print(df.head())

## 2. 数据概览

使用统计方法快速了解数据的基本特征。

In [ ]:
# 基本统计信息
print("数值型变量的统计摘要:")
print(df.describe())

In [ ]:
# 数据类型和缺失值
print("\n数据类型:")
print(df.dtypes)

print("\n缺失值统计:")
print(df.isnull().sum())

In [ ]:
# 等级分布
grade_counts = df['等级'].value_counts().sort_index()
print("等级分布:")
print(grade_counts)

# 可视化等级分布
plt.figure(figsize=(10, 6))
grade_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('学生等级分布', fontsize=14)
plt.xlabel('等级', fontsize=12)
plt.ylabel('人数', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

## 3. 单变量分析

分析单个变量的分布特征。

In [ ]:
# 各科成绩的分布
subjects = ['数学成绩', '物理成绩', '化学成绩', '英语成绩']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, subject in enumerate(subjects):
    axes[i].hist(df[subject], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[i].axvline(df[subject].mean(), color='red', linestyle='--', linewidth=2, label=f'均值: {df[subject].mean():.1f}')
    axes[i].axvline(df[subject].median(), color='green', linestyle='--', linewidth=2, label=f'中位数: {df[subject].median():.1f}')
    axes[i].set_title(f'{subject}分布', fontsize=12)
    axes[i].set_xlabel('分数', fontsize=10)
    axes[i].set_ylabel('频数', fontsize=10)
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 箱线图比较各科成绩
plt.figure(figsize=(12, 6))
df[subjects].boxplot()
plt.title('各科成绩箱线图', fontsize=14)
plt.ylabel('分数', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

In [ ]:
# 学习时间和出勤率的分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 学习时间
axes[0].hist(df['学习时间'], bins=20, edgecolor='black', alpha=0.7, color='coral')
axes[0].set_title('学习时间分布', fontsize=12)
axes[0].set_xlabel('小时/周', fontsize=10)
axes[0].set_ylabel('频数', fontsize=10)
axes[0].grid(True, alpha=0.3)

# 出勤率
axes[1].hist(df['出勤率'], bins=20, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1].set_title('出勤率分布', fontsize=12)
axes[1].set_xlabel('出勤率', fontsize=10)
axes[1].set_ylabel('频数', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 双变量分析

分析两个变量之间的关系。

In [ ]:
# 成绩与学习时间的关系
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, subject in enumerate(subjects):
    axes[i].scatter(df['学习时间'], df[subject], alpha=0.6, s=50)
    
    # 添加趋势线
    z = np.polyfit(df['学习时间'], df[subject], 1)
    p = np.poly1d(z)
    axes[i].plot(df['学习时间'], p(df['学习时间']), "r--", alpha=0.8, linewidth=2)
    
    # 计算相关系数
    corr = df['学习时间'].corr(df[subject])
    axes[i].set_title(f'{subject} vs 学习时间 (r={corr:.3f})', fontsize=12)
    axes[i].set_xlabel('学习时间 (小时/周)', fontsize=10)
    axes[i].set_ylabel('分数', fontsize=10)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 出勤率与平均分的关系
plt.figure(figsize=(10, 6))
plt.scatter(df['出勤率'], df['平均分'], alpha=0.6, s=50, c='purple')

# 添加趋势线
z = np.polyfit(df['出勤率'], df['平均分'], 1)
p = np.poly1d(z)
plt.plot(df['出勤率'], p(df['出勤率']), "r--", alpha=0.8, linewidth=2)

corr = df['出勤率'].corr(df['平均分'])
plt.title(f'出勤率 vs 平均分 (r={corr:.3f})', fontsize=14)
plt.xlabel('出勤率', fontsize=12)
plt.ylabel('平均分', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

## 5. 多变量分析

分析多个变量之间的关系。

In [ ]:
# 相关性矩阵
numeric_cols = subjects + ['学习时间', '出勤率', '平均分']
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, fmt='.3f')
plt.title('相关性矩阵', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 散点图矩阵
from pandas.plotting import scatter_matrix

# 选择部分变量进行可视化
plot_cols = ['数学成绩', '物理成绩', '化学成绩', '平均分']

scatter_matrix(df[plot_cols], figsize=(12, 12), alpha=0.6, diagonal='hist',
               grid=True, edgecolor='black')
plt.suptitle('成绩散点图矩阵', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

In [ ]:
# 不同等级的学习时间和出勤率对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 学习时间
df.boxplot(column='学习时间', by='等级', ax=axes[0])
axes[0].set_title('不同等级的学习时间', fontsize=12)
axes[0].set_xlabel('等级', fontsize=10)
axes[0].set_ylabel('学习时间 (小时/周)', fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# 出勤率
df.boxplot(column='出勤率', by='等级', ax=axes[1])
axes[1].set_title('不同等级的出勤率', fontsize=12)
axes[1].set_xlabel('等级', fontsize=10)
axes[1].set_ylabel('出勤率', fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')

plt.suptitle('')  # 移除自动生成的标题
plt.tight_layout()
plt.show()

## 6. 高级可视化

使用更复杂的可视化技术展示数据。

In [ ]:
# 三维散点图
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# 根据等级设置颜色
grade_colors = {'A': 'red', 'B': 'orange', 'C': 'yellow', 'D': 'green', 'F': 'blue'}
colors = df['等级'].map(grade_colors)

scatter = ax.scatter(df['数学成绩'], df['物理成绩'], df['化学成绩'], 
                     c=colors, s=50, alpha=0.6)

ax.set_xlabel('数学成绩', fontsize=12)
ax.set_ylabel('物理成绩', fontsize=12)
ax.set_zlabel('化学成绩', fontsize=12)
ax.set_title('三科成绩三维散点图', fontsize=14)

# 添加图例
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=grade,
                          markerfacecolor=color, markersize=10)
                   for grade, color in grade_colors.items()]
ax.legend(handles=legend_elements, loc='upper left')

plt.show()

In [ ]:
# 小提琴图
plt.figure(figsize=(12, 6))

# 重塑数据以适应小提琴图
melted_df = df[subjects].melt(var_name='科目', value_name='分数')

sns.violinplot(x='科目', y='分数', data=melted_df, palette='Set2')
plt.title('各科成绩小提琴图', fontsize=14)
plt.xlabel('科目', fontsize=12)
plt.ylabel('分数', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

In [ ]:
# 配对图
plt.figure(figsize=(12, 12))
sns.pairplot(df[subjects + ['等级']], hue='等级', palette='Set2',
             plot_kws={'alpha': 0.6, 's': 50, 'edgecolor': 'k'})
plt.suptitle('成绩配对图', fontsize=14, y=1.02)
plt.show()

## 7. 统计检验

使用统计方法验证观察到的差异是否显著。

In [ ]:
# 比较不同等级的学习时间
from scipy.stats import f_oneway

# 按等级分组
groups = [df[df['等级'] == grade]['学习时间'] for grade in ['A', 'B', 'C', 'D', 'F']]
groups = [g for g in groups if len(g) > 0]  # 移除空组

# 单因素方差分析
f_stat, p_value = f_oneway(*groups)

print("单因素方差分析结果:")
print(f"F统计量: {f_stat:.4f}")
print(f"p值: {p_value:.4f}")

if p_value < 0.05:
    print("结论: 不同等级的学习时间存在显著差异")
else:
    print("结论: 不同等级的学习时间无显著差异")

In [ ]:
# 相关性显著性检验
from scipy.stats import pearsonr

print("相关性检验结果:")
print("-" * 50)

for subject in subjects:
    corr, p_val = pearsonr(df['学习时间'], df[subject])
    significance = "显著" if p_val < 0.05 else "不显著"
    print(f"{subject}与学习时间:")
    print(f"  相关系数: {corr:.4f}")
    print(f"  p值: {p_val:.4f}")
    print(f"  显著性: {significance}")
    print()

## 8. 总结与洞察

基于以上分析，我们可以得出以下结论：

1. **成绩分布**: 各科成绩大致呈正态分布，数学和物理的平均分较高。

2. **学习时间的影响**: 学习时间与各科成绩呈正相关，学习时间越长，成绩越好。

3. **出勤率的影响**: 出勤率与平均分呈正相关，出勤率高的学生平均分更高。

4. **科目相关性**: 数理化三科之间有较强的相关性，说明理科基础对成绩有重要影响。

5. **等级差异**: 不同等级的学生在学习时间和出勤率上存在显著差异。

## 9. 练习题

### 练习1
创建一个新的数据集，分析不同班级学生的成绩分布，并比较班级之间的差异。

In [ ]:
# 你的代码


### 练习2
使用Plotly库创建交互式可视化图表，展示成绩与学习时间的关系。

In [ ]:
# 你的代码


### 练习3
进行更深入的统计分析，如回归分析，预测学生的平均分。

In [ ]:
# 你的代码
